# 트랜스포머 (Transformer)

- Attention Mechanism을 응용하여, 이전에 RNN이나 LSTM 모델에서 Seq 단위의 데이터를 순차적으로 처리하는것과 달리, 전체 Seq를 한번에 처리하는 모델 

![image1](https://wikidocs.net/images/page/31379/transformer_attention_overview.PNG)

- 구성요소 : 
    - Self - Attention :
        - 하나의 Seq내에서 각 위치의 단어가 Seq내 다른 위치의 단어라 얼마나 관련이 있는지 계산 
        - 모델이 문장 내에서 각 단어간의 관계가 어떻게 되는지를 파악 
    - Muti Head Attention : 
        -  Self - Attention 모델을 병렬로 반복 수행하면서 다양한 표현에서 정보를 추출     
    - Position Encoding :
        - 전체 Seq를 한꺼번에 처리하기 때문에, 순서정보가 소실될 가능성이 매우 높음 
        - 이를 보완하기 위해 Sequence의 순서정보를 가진 Position Encoding을 수행 

In [4]:
# !pip install tensorflow_datasets

In [2]:
import pandas as pd 
import numpy as np
import re 
import tensorflow_datasets as tfds 

In [3]:
df1 = pd.read_csv('39_Data.csv')
print(df1.shape)
df1.head()

(11823, 4)


,Unnamed: 0,Q,A,label
0,0,12시 땡!,하루가 또 가네요.,0
1,1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,4,PPL 심하네,눈살이 찌푸려지죠.,0


In [5]:
# 특수문자에 대해 띄어쓰기 
# 특수문자 앞뒤를 기준으로 공백을 추가 (?!.,)
df1['Q']

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [9]:
questions = [re.sub(r"([?.!,])", r" \1 ", x).strip() for x in df1['Q']]

In [11]:
questions[:10]

['12시 땡 !',
 '1지망 학교 떨어졌어',
 '3박4일 놀러가고 싶다',
 '3박4일 정도 놀러가고 싶다',
 'PPL 심하네',
 'SD카드 망가졌어',
 'SD카드 안돼',
 'SNS 맞팔 왜 안하지ㅠㅠ',
 'SNS 시간낭비인 거 아는데 매일 하는 중',
 'SNS 시간낭비인데 자꾸 보게됨']

In [12]:
answers = [re.sub(r"([?.!,])", r" \1 ", x).strip() for x in df1['A']]
answers[:10]

['하루가 또 가네요 .',
 '위로해 드립니다 .',
 '여행은 언제나 좋죠 .',
 '여행은 언제나 좋죠 .',
 '눈살이 찌푸려지죠 .',
 '다시 새로 사는 게 마음 편해요 .',
 '다시 새로 사는 게 마음 편해요 .',
 '잘 모르고 있을 수도 있어요 .',
 '시간을 정하고 해보세요 .',
 '시간을 정하고 해보세요 .']

- 서브토크나이저 : 훈련 데이터에 없는 새로운 단어가 등장하더라도, (OOV, Out of Vocabulary)  그 단어를 구성하는 서브 워드를 분해하여 처리 

In [16]:
# 서브워드 텍스트 인코더 함수를 활용해, 질문/답변에 있는 서브 워드를 정리 
tokenizer= tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
                                    questions + answers, target_vocab_size=2**13)

In [17]:
# 만들어진 토큰 개수 
tokenizer.vocab_size

8178

In [18]:
# 처리 된 토큰 정보 
tokenizer.subwords

[' .',
 ' ?',
 '거예요',
 '수_',
 '게_',
 '너무_',
 '더_',
 '거_',
 '좋아하는_',
 '는_',
 '이_',
 '을_',
 '잘_',
 '도_',
 ' .  ',
 '고_',
 '요',
 '것_',
 '많이_',
 '안_',
 '좋은_',
 '같아요',
 '한_',
 '좀_',
 '있어요',
 '싶어',
 '가_',
 '나_',
 '에_',
 '있을_',
 '지_',
 '해보세요',
 '은_',
 '사람_',
 '할_',
 '해',
 '같아',
 '네',
 '면_',
 '건_',
 ' !',
 '사람이_',
 '를_',
 '마세요',
 '다_',
 '하고_',
 '지',
 '하는_',
 '보세요',
 '죠',
 '어',
 '서_',
 '내가_',
 '의_',
 '다',
 '내_',
 '이제_',
 '마음이_',
 '나',
 '다른_',
 '썸_',
 '만_',
 '그_',
 '어떻게_',
 '있는_',
 '왜_',
 '싶다',
 '세요',
 '다시_',
 '시간이_',
 '수도_',
 '없어',
 '것도_',
 '또_',
 '좋을_',
 '오늘_',
 '정말_',
 '가',
 '이',
 '같이_',
 '네요',
 '될_',
 '해요',
 '자꾸_',
 '걸_',
 '있어',
 '하세요',
 '없어요',
 '일_',
 '제가_',
 '길_',
 '바랄게요',
 '로_',
 '까',
 '돼요',
 '하면_',
 '봐요',
 '할까',
 '때_',
 '저도_',
 '으로_',
 '먼저_',
 '있을까',
 '에서_',
 '주세요',
 '그런_',
 '헤어진지_',
 '이별_',
 '될까',
 '기_',
 '고',
 '진짜_',
 '나를_',
 '야',
 '마음을_',
 '여자친구가_',
 '기',
 '좋아요',
 '없는_',
 '계속_',
 '남자친구가_',
 '혼자_',
 '해도_',
 '이별',
 '못_',
 '드세요',
 '줄_',
 '않아요',
 '먹고_',
 '는데_',
 '좋죠',
 '하지_',
 '일이_',
 '힘든

In [23]:
# <SOS> : 문장의 시작을 알리는 토큰 / <EOS> : 문장의 끝을 알리는 토큰 
# 시작 토큰과 종료토큰을 서브토크나이저에 추가 
START_TOKEN = tokenizer.vocab_size
END_TOKEN   = tokenizer.vocab_size + 1 
# 사전의 크기도 2개 증가 (SOS/ EOS 추가)
VOCAB_SIZE = tokenizer.vocab_size + 2 

In [27]:
tokenizer.encode(questions[5])

[8005, 7990, 2192, 919, 78, 821]

In [28]:
token_list = [5766, 611, 3509, 141, 685, 3747, 849]
tokenizer.decode(token_list)

'가스비 비싼데 감기 걸리겠어'

- 전체 데이터를 이용한 Text To Sequence와 Padding 작업을 실시 

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [36]:
# 토큰화 / 인코딩 / 시작,종료토큰 추가 / 패딩 
def tokenize_and_filer(text):
    text_list = [ ]
    for i in text: 
        # SOS / EOS을 추가하여, 토큰화 실시 
        sent1 = [START_TOKEN] + tokenizer.encode(i) + [END_TOKEN]
        text_list.append(sent1)
    return pad_sequences(text_list, maxlen=40, padding='post')

In [38]:
questions_out = tokenize_and_filer(questions)
answers_out   = tokenize_and_filer(answers)

In [39]:
questions_out

array([[8178, 7915, 4207, ...,    0,    0,    0],
       [8178, 7971,   47, ...,    0,    0,    0],
       [8178, 7973, 1435, ...,    0,    0,    0],
       ...,
       [8178, 8159, 8079, ...,    0,    0,    0],
       [8178,  134,  166, ...,    0,    0,    0],
       [8178, 1953,  884, ...,    0,    0,    0]])

In [40]:
answers_out

array([[8178, 3844,   74, ...,    0,    0,    0],
       [8178, 1830, 5502, ...,    0,    0,    0],
       [8178, 3400,  777, ...,    0,    0,    0],
       ...,
       [8178, 5211,  287, ...,    0,    0,    0],
       [8178,   13, 3201, ...,    0,    0,    0],
       [8178,  221,  555, ...,    0,    0,    0]])

- Teacher Forcing (교사강제) : 모델의 현재 출력을 다음 시점의 입력으로 사용하는 대신, 실제 목표 문장의 현재 시점 데이터를 다음 시점의 입력으로 사용

In [41]:
from tensorflow.data import Dataset
from tensorflow.data.experimental import AUTOTUNE

In [42]:
# Decode 각 시작점에서 다음 토큰을 본래 데이터 셋을 참조해 교사강제를 수행하도록 
# 디코더에 <SOS> / <EOS> 제거 한 뒤 처리 
dataset = Dataset.from_tensor_slices((
    # 디코더의 입력값 
    {'inputs':questions_out, 'dec_inputs': answers_out[: , :-1]},
    {'outputs':answers_out[:, 1:]},
))

In [44]:
# 데이터 셋을 빠르게 로드해 Epoch 수행 
dataset1 = dataset.cache()
# 데이터를 batch size로 묶음 
dataset2 = dataset1.batch(64)
dataset3 = dataset2.prefetch(AUTOTUNE)

In [45]:
dataset3

<_PrefetchDataset element_spec=({'inputs': TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(None, 39), dtype=tf.int32, name=None)}, {'outputs': TensorSpec(shape=(None, 39), dtype=tf.int32, name=None)})>

# 포지셔널 인코딩 (Positional Encoding) 

- Transformer의 입력층으로 사용되는 층 
- 기본적인 RNN 계열의 모델은 단어를 순차적으로 받아 처리하기 때문에, 문장 내 단어의 순서가 처뢰든 정보가 존재 
![image1](https://wikidocs.net/images/page/31379/transformer2.PNG)

- Transformer모델이 문장을 한꺼번에 처리 하기 때문에, 문장 내 단어 순서의 정보가 소실 될 가능성이 매우 높음
- 이를 Positional Encoding 기법을 활용해 각 단어의 위치정보를 준 Vector를 계산하여 학습을 수행 
![image1](https://wikidocs.net/images/page/31379/transformer6_final.PNG)

- 사인함수와 코사인함수를 활용해 단어의 순서정보를 벡터 형태로 변환 / 단어 정보와 같이 Encoder와 Decoder에 전달 

In [46]:
from keras.layers import Layer

In [47]:
import tensorflow as tf 

In [48]:
class PositionalEncoding(Layer):
    # 해당 클래스를 만들때 생성되는 객체의 초기 정보가 들어간 함수 구성 
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)
    # position : 문장의 최대 길이를 지정(해당 문장의 위치정보가 들어가는 벡터의 크기) 
    # d_model : 벡터를 구성해줄 차원 수 
    
    # i 각 차원의 인덱스를 나타내는 정보 
    # 각 단어에 대한 위치 정보를 Cos 함수와 Sin 로 계산하기 위해 
    # 각 벡터에 각도 정보를 계산 
    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles
    
    # 앞서 계산한 각도를 이용해 Position Vector를 구성 
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            # 각 위치에 대한 인덱스를 계산해 벡터로 변환 
            position = tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            # 각 차원의 인덱스를 계산해 벡터로 변환 
            i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, : ],
            d_model=d_model)
        
        # 각 배열의 짝수에 해당하는 인덱스에는 Sin함수 적용 
        sines = tf.math.sin(angle_rads[:, 0::2])
        # 각 배열의 홀수에 해당하는 인덱스에는 Cos함수 적용 
        cosines = tf.math.cos(angle_rads[:, 1::2])
        
        # 앞서 계산한 코사인 / 사인함수의 결과를 벡터로 구성 
        # 짝수 위치에는 sin함수 결과 / 홀수 위치에는 cos함수 결과 
        angle_rads = np.zeros(angle_rads.shape)
        angle_rads[:, 0::2] = sines
        angle_rads[:, 1::2] = cosines
        pos_encoding = tf.constant(angle_rads)
        # 문장 길이가 변화할 때 벡터의 크기도 유연하게 변화되게끔 구성
        pos_encoding = pos_encoding[tf.newaixs, ...]

        return tf.cast(pos_encoding, tf.float32)
    
    # 앞서 만들어진 포지션 벡터를 모델 내에서 토큰 행렬과 함께 인식되도록 입력 
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], : ]
# 1. 먼저 모델에서 입력 데이터를 받음
# 2, 받은 데이터를 포지션 인코딩을 통해 각 위치벡터를 계산 
# 3. 입력 위치에 인코딩으로 계산된 벡터 값을 더해 위치 정보를 유지 